In [1]:
import urllib.request, urllib.parse, urllib.error
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import wget
from pandas.io.json import json_normalize
import requests 

import numpy as np # library to handle data in a vectorized manner
import urllib.request, json 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# we use Beautiful soup for parsing the wiki page
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# look for the table we need, with class wikitable sortable
table_w = soup.find('table','wikitable sortable')

In [5]:
# look for the cell headers with <th> element
th_1 = table_w.tr.find_all("th")
titulos = list()
for element in th_1:
    element = str(element)
    element = element.replace('<th>','').replace('</th>','').replace('\n','')
    titulos.append(element)
titulos

['Postal Code', 'Borough', 'Neighborhood']

In [6]:
# look for the table data with <tr> 
th_2 = table_w.find_all("tr")
# we delete the first list that contains the headers
th_2 = th_2[1:]
# convert the html code to text str, we edit the text we dont need "<tr>, </tr> and \n". We split each row with <td> 
# select the only elements in the list we are interested
data = list()
for lista in th_2:
    lista = str(lista)
    lista = lista.replace('<tr>','').replace('</tr>','').replace('</td>','').replace('\n','')
    lista = lista.split('<td>')
    lista = lista[1:4]
    data.append(lista)

In [7]:
# we convert list to a dataframe
data_1 = pd.DataFrame(data)
# add the titles and delete a 4th column generated by error
data_1.columns = titulos
# we drop Borough which are not assigned
data_1.set_index('Borough', inplace=True) 
data_1.drop(["Not assigned"], inplace = True) 
data_1.reset_index(inplace=True)

In [8]:
# control if we have a 'Not assigned' Neighborhood
data_1_no = data_1[data_1['Neighborhood']=='Not assigned']
print(data_1_no.shape)

(0, 3)


In [9]:
# check 
data_1.shape

(103, 3)

In [10]:
# check the look
data_1.set_index('Postal Code', inplace=True) 
data_1

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [12]:
# Download latitude and longitude data
url = 'https://cocl.us/Geospatial_data'
filename = wget.download(url)   

100% [................................................................................] 2891 / 2891

In [14]:
# Edit data
location_2 = pd.read_csv(filename, delimiter=",")
location_2.set_index('Postal Code', inplace=True)
location_2

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [25]:
# Join geolocation data with postal codes table
data_2 = data_1.join(location_2)
data_2.shape

(103, 4)

In [17]:
# control if we have a 'Not assigned' Latitude or Longitude
data_2_no = data_2[data_2['Latitude']=='Not assigned']
print(data_2_no.shape)
data_2_no = data_2[data_2['Longitude']=='Not assigned']
print(data_2_no.shape)
# Show postal codes with geolocation with index reseted
data_2.reset_index(inplace=True)
data_2.head()

(0, 4)
(0, 4)


C:\Users\juan_\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = '5VJ44BEITBESYOZPPQVOENNO4QW2T22FCLL5OOWDHGK2RG0Z' # your Foursquare ID
CLIENT_SECRET = 'CSBGMT3XQ3GN0OJIPYXYTKBLX2NAP42OWFTMZM1BIO4ZLCCB' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5VJ44BEITBESYOZPPQVOENNO4QW2T22FCLL5OOWDHGK2RG0Z
CLIENT_SECRET:CSBGMT3XQ3GN0OJIPYXYTKBLX2NAP42OWFTMZM1BIO4ZLCCB


#### Function to process all the neighborhoods

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Funcion get_category_type from the Foursquare lab.

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [30]:
# look for venues in each neighborhood
toronto_venues = getNearbyVenues(names=data_2['Neighborhood'],
                                   latitudes=data_2['Latitude'],
                                   longitudes=data_2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Check the size of the resulting dataframe

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(2132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [123]:
# check unique categories values. 
venues_list = toronto_venues['Venue Category'].value_counts().rename_axis('unique_values').to_frame('counts')
venues_list.reset_index(inplace=True)
venues_list

,unique_values,counts
0,Restaurant,484
1,Coffee Shop,289
2,Bar,143
3,Gym,53
4,Pizza Place,51
5,Park,49
6,Sandwich Place,42
7,Bakery,41
8,Hotel,38
9,Clothing Store,34


In [76]:
#unify categories 'cafe' with 'coffee shop' and Tea Room which are essentially the same
toronto_venues['Venue Category'].replace(to_replace='Café', value='Coffee Shop', inplace=True)
toronto_venues['Venue Category'].replace(to_replace='Tea Room', value='Coffee Shop', inplace=True)

In [88]:
#unify all type of Restaurants categories
toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Restaurant'), 'Venue Category'] = 'Restaurant'

In [91]:
#unify all type of Bar / Pub categories
toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Bar'), 'Venue Category'] = 'Bar'
toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Pub'), 'Venue Category'] = 'Bar'
toronto_venues['Venue Category'].replace(to_replace='Gastropub', value='Bar', inplace=True)

In [92]:
#unify all type of Gym categories
toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Gym'), 'Venue Category'] = 'Gym'

In [122]:
#unify all type of Airport services
toronto_venues.loc[toronto_venues['Venue Category'].str.contains('Airport'), 'Venue Category'] = 'Airport'

In [124]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.shape

(2132, 207)

#### Goup rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [125]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Camera Store,Candy Store,Cheese Shop,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Arts Building,College Auditorium,College Rec Center,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Farmers Market,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Intersection,Jazz Club,Jewelry Store,Kids Store,Lake,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tennis Court,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Video Game Store,Video Store,Warehouse Store,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

#### Confirm the new size

In [126]:
toronto_grouped.shape

(95, 207)

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [141]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Restaurant,Women's Store,Department Store,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Sandwich Place,Bar,Gym,Arts & Crafts Store,Concert Hall
2,"Bathurst Manor, Wilson Heights, Downsview North",Restaurant,Bank,Coffee Shop,Fried Chicken Joint,Shopping Mall,Diner,Sandwich Place,Deli / Bodega,Ice Cream Shop,Mobile Phone Shop
3,Bayview Village,Restaurant,Coffee Shop,Bank,Women's Store,Department Store,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Bar,Sandwich Place,Butcher,Grocery Store,Pizza Place,Pharmacy,Liquor Store,Hobby Shop


### CLUSTER NEIGHBORHOODS

In [104]:
from sklearn.cluster import KMeans 

Run *k*-means to cluster the neighborhood into 5 clusters.

In [128]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1])

New dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [129]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data_2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.shape

(103, 15)

In [130]:
# check if we have nulls values in any neighboorhoods
toronto_merged.isnull().values.any()
toronto_merged.isnull().sum().sum()

44

In [131]:
# delete few neighboorhoods with errors
toronto_merged.dropna(inplace = True)
toronto_merged.shape

(99, 15)

In [143]:
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,1,Fireworks Store,Park,Food & Drink Shop,Department Store,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run,Distribution Center
M4A,North York,Victoria Village,43.725882,-79.315572,0,Grocery Store,Coffee Shop,Hockey Arena,Restaurant,Deli / Bodega,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bar,Restaurant,Bakery,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Coffee Shop,Restaurant,Boutique,Miscellaneous Shop,Event Space,Gift Shop,Accessories Store,Donut Shop
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Restaurant,Bar,Yoga Studio,Fried Chicken Joint,Park,College Auditorium,Music Venue,Creperie,Diner


In [133]:
# convert cluster labels to int datatype
toronto_merged = toronto_merged.astype({"Cluster Labels": int})

In [134]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Finally, let's visualize the resulting clusters

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Construction & Landscaping,Women's Store,Deli / Bodega,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run,Distribution Center
M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,4,Park,Playground,Women's Store,Deli / Bodega,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run,Distribution Center
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,Women's Store,Dance Studio,Electronics Store,Drugstore,Donut Shop,Dog Run,Distribution Center


In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M9M,North York,"Humberlea, Emery",43.724766,-79.532242,3,Food Service,Baseball Field,Women's Store,Dessert Shop,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3,Baseball Field,Women's Store,Fireworks Store,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run,Distribution Center
